In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
import seaborn as sns
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.callbacks import LearningRateScheduler
import matplotlib.pyplot as plt

In [2]:
dataset_dir = './garbage_classification/'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

In [ ]:
val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
val_generator = val_datagen.flow_from_directory(
    dataset_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # Use the validation split
)

In [5]:
dim = 150  # Input image dimensions
classes = list(train_generator.class_indices.keys())  # Class labels
initial_learning_rate = 1e-4
epochs_1 = 30
steps_per_epoch = train_generator.samples // train_generator.batch_size
validation_steps = val_generator.samples // val_generator.batch_size


In [ ]:
classes

In [ ]:
conv_base = DenseNet121(weights='imagenet', include_top=False, input_shape=(dim, dim, 3))
conv_base.trainable = False 

In [8]:
from tensorflow.keras.layers import GlobalAveragePooling2D


In [9]:
model = Sequential([
    conv_base,
    GlobalAveragePooling2D(),
    Dropout(0.5),  # Regularization to prevent overfitting
    Dense(len(classes), activation='softmax')  # Output layer for classification
])


In [10]:
from tensorflow.keras import  optimizers


In [11]:
model.compile(
    optimizer=optimizers.Adam(learning_rate=initial_learning_rate),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
model.summary()


In [13]:
def lr_step_decay(epoch, lr):
    drop_rate = 0.5
    epochs_drop = 10
    return lr * drop_rate if (epoch % epochs_drop == 0 and epoch > 0) else lr

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs_1,
    validation_data=val_generator,
    validation_steps=validation_steps,
    callbacks=[LearningRateScheduler(lr_step_decay, verbose=1)]  # Optional learning rate scheduler
)


In [15]:
def plot_metrics(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs_range = range(len(acc))

    plt.figure(figsize=(12, 6))

    # Accuracy plot
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    # Loss plot
    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')

    plt.show()

In [ ]:
plot_metrics(history)

In [ ]:
# Save the model
model.export('waste_sorting_model')


In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model('waste_sorting_model')
tflite_model = converter.convert()

# Save the TensorFlow Lite model
with open('waste_sorting_model.tflite', 'wb') as f:
    f.write(tflite_model)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
test_generator = test_datagen.flow_from_directory(
    dataset_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # Use the validation split
)
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy:.2f}")


In [ ]:
y_pred = model.predict(val_generator)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = val_generator.classes

# Confusion matrix and classification report
print(confusion_matrix(y_true, y_pred_classes))
print(classification_report(y_true, y_pred_classes, target_names=val_generator.class_indices.keys()))

In [21]:
def display_predictions(generator, model, num_images=9):
    x_batch, y_batch = next(generator)
    predictions = model.predict(x_batch)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(y_batch, axis=1)

    plt.figure(figsize=(12, 12))
    for i in range(num_images):
        plt.subplot(3, 3, i + 1)
        plt.imshow(x_batch[i])
        plt.axis('off')
        true_label = list(generator.class_indices.keys())[true_classes[i]]
        predicted_label = list(generator.class_indices.keys())[predicted_classes[i]]
        color = 'green' if true_label == predicted_label else 'red'
        plt.title(f"True: {true_label}\nPred: {predicted_label}", color=color)
    plt.tight_layout()
    plt.show()

In [ ]:
display_predictions(val_generator, model)


In [23]:
def plot_confusion_matrix(y_true, y_pred_classes, class_labels):
    cm = confusion_matrix(y_true, y_pred_classes)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.title('Confusion Matrix')
    plt.show()


In [ ]:
plot_confusion_matrix(y_true, y_pred_classes, list(val_generator.class_indices.keys()))
